## 1) Let's prepare the scripts to be deployed as a Glue Job

In [ ]:
%%bash
mkdir /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample
wget https://raw.githubusercontent.com/angelocarvalho/glue-python-shell-sample/master/glue-python-shell-sample-whl/setup.py

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample
python3 setup.py bdist_wheel

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample/dist
wget https://raw.githubusercontent.com/angelocarvalho/glue-python-shell-sample/master/etl_with_pandas.py

In [ ]:
import simplejson
with open('/opt/ml/metadata/resource-metadata.json') as fh:
    metadata = simplejson.loads(fh.read())
accountid = metadata['ResourceArn'].split(':')[4]

%set_env accountid={accountid}
%set_env bucket_name=lab-{accountid}

## 2) Open the file etl_with_pandas.py and add your bucket name!

In [ ]:
%%bash
aws s3 cp /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample/dist/glue_python_shell_sample_module-0.1-py3-none-any.whl s3://$bucket_name/lib/
aws s3 cp /home/ec2-user/SageMaker/Analytics_Labs/glue_python_shell_sample/dist/etl_with_pandas.py s3://$bucket_name/scripts/

## 3) Replace the iam role and and the bucket name before running the command bellow

In [ ]:
%%bash
aws glue create-job --name etl_with_pandas \
    --role <<nome_do_seu_iam_role>> \
    --command '{"Name" :  "pythonshell", "PythonVersion" : "3", "ScriptLocation" : "s3://<<nome_do_seu_bucket>>/scripts/etl_with_pandas.py"}' \
    --default-arguments '{"--extra-py-files" : "s3://<<nome_do_seu_bucket>>/lib/glue_python_shell_sample_module-0.1-py3-none-any.whl"}'